In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
from citipy import citipy

In [4]:
countries = 'https://api.covid19api.com/countries'

In [7]:
response_countries = requests.get(countries).json()
pprint(response_countries)

[{'Country': 'Argentina', 'ISO2': 'AR', 'Slug': 'argentina'},
 {'Country': 'Dominica', 'ISO2': 'DM', 'Slug': 'dominica'},
 {'Country': 'Pitcairn', 'ISO2': 'PN', 'Slug': 'pitcairn'},
 {'Country': 'Greece', 'ISO2': 'GR', 'Slug': 'greece'},
 {'Country': 'Madagascar', 'ISO2': 'MG', 'Slug': 'madagascar'},
 {'Country': 'Papua New Guinea', 'ISO2': 'PG', 'Slug': 'papua-new-guinea'},
 {'Country': 'Vanuatu', 'ISO2': 'VU', 'Slug': 'vanuatu'},
 {'Country': 'Mongolia', 'ISO2': 'MN', 'Slug': 'mongolia'},
 {'Country': 'Bulgaria', 'ISO2': 'BG', 'Slug': 'bulgaria'},
 {'Country': 'Central African Republic',
  'ISO2': 'CF',
  'Slug': 'central-african-republic'},
 {'Country': 'Cook Islands', 'ISO2': 'CK', 'Slug': 'cook-islands'},
 {'Country': 'Rwanda', 'ISO2': 'RW', 'Slug': 'rwanda'},
 {'Country': 'Virgin Islands, US', 'ISO2': 'VI', 'Slug': 'virgin-islands'},
 {'Country': 'Austria', 'ISO2': 'AT', 'Slug': 'austria'},
 {'Country': 'Chile', 'ISO2': 'CL', 'Slug': 'chile'},
 {'Country': 'Ethiopia', 'ISO2': 'ET

store response_countries in to a df.
Import csv into a df.
parse the country name column to add hyphens replacing spaces.
do a lookup on response_countries df to get the slugs.
create for loop to check all the slugs and make separate api calls.

In [2]:
search_url = 'https://api.covid19api.com/total/country/india'

In [3]:
response = requests.get(search_url).json()
pprint(response)

[{'Active': 0,
  'City': '',
  'CityCode': '',
  'Confirmed': 0,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-01-22T00:00:00Z',
  'Deaths': 0,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 0},
 {'Active': 0,
  'City': '',
  'CityCode': '',
  'Confirmed': 0,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-01-23T00:00:00Z',
  'Deaths': 0,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 0},
 {'Active': 0,
  'City': '',
  'CityCode': '',
  'Confirmed': 0,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-01-24T00:00:00Z',
  'Deaths': 0,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 0},
 {'Active': 0,
  'City': '',
  'CityCode': '',
  'Confirmed': 0,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-01-25T00:00:00Z',
  'Deaths': 0,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 0},
 {'Active': 0,
  'City': '',
  'CityCode': '',
  'Confirmed': 0,
  'Country': 'India',
  'CountryCode': '',
  'D

 {'Active': 24641,
  'City': '',
  'CityCode': '',
  'Confirmed': 34863,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-04-30T00:00:00Z',
  'Deaths': 1154,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 9068},
 {'Active': 26027,
  'City': '',
  'CityCode': '',
  'Confirmed': 37257,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-05-01T00:00:00Z',
  'Deaths': 1223,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 10007},
 {'Active': 27557,
  'City': '',
  'CityCode': '',
  'Confirmed': 39699,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-05-02T00:00:00Z',
  'Deaths': 1323,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 10819},
 {'Active': 29339,
  'City': '',
  'CityCode': '',
  'Confirmed': 42505,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-05-03T00:00:00Z',
  'Deaths': 1391,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 11775},
 {'Active': 32024,
  'City': '',
  'CityCode': '',
  

 {'Active': 628747,
  'City': '',
  'CityCode': '',
  'Confirmed': 2153010,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-08-08T00:00:00Z',
  'Deaths': 43379,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 1480884},
 {'Active': 634945,
  'City': '',
  'CityCode': '',
  'Confirmed': 2215074,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-08-09T00:00:00Z',
  'Deaths': 44386,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 1535743},
 {'Active': 639929,
  'City': '',
  'CityCode': '',
  'Confirmed': 2268675,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-08-10T00:00:00Z',
  'Deaths': 45257,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 1583489},
 {'Active': 643948,
  'City': '',
  'CityCode': '',
  'Confirmed': 2329638,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-08-11T00:00:00Z',
  'Deaths': 46091,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 1639599},
 {'Active': 653622,
  'City'

 {'Active': 465478,
  'City': '',
  'CityCode': '',
  'Confirmed': 8845127,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-11-15T00:00:00Z',
  'Deaths': 130070,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 8249579},
 {'Active': 453401,
  'City': '',
  'CityCode': '',
  'Confirmed': 8874290,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-11-16T00:00:00Z',
  'Deaths': 130519,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 8290370},
 {'Active': 446805,
  'City': '',
  'CityCode': '',
  'Confirmed': 8912907,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-11-17T00:00:00Z',
  'Deaths': 130993,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 8335109},
 {'Active': 443303,
  'City': '',
  'CityCode': '',
  'Confirmed': 8958483,
  'Country': 'India',
  'CountryCode': '',
  'Date': '2020-11-18T00:00:00Z',
  'Deaths': 131578,
  'Lat': '0',
  'Lon': '0',
  'Province': '',
  'Recovered': 8383602},
 {'Active': 443794,
  'C

>>> import re
>>> s = "How are you"
>>> print re.sub('\s', '-', s)